<a href="https://colab.research.google.com/github/ooodmt/MLMIP/blob/master/Kopie_von_efficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import time
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import random
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torch.autograd import Variable
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from torch.utils.tensorboard import SummaryWriter 
from efficientnet_pytorch import EfficientNet

ModuleNotFoundError: ignored

In [ ]:
valid_csv_path = '/home/tu-serbin/data/chexpert/v1.0/valid.csv'
model_save_dir = '/home/tu-serbin/group/igloo/alex/saves/'

dir_path = '/home/tu-serbin/data/chexpert/v1.0/'
data_path = '/home/tu-serbin/data/'

cams_path = '/home/tu-serbin/group/igloo/alex/CAMs/'
comp_path = '/home/tu-serbin/group/igloo/alex/CAMs/Compound/'


In [ ]:
efnet = EfficientNet2()

load2 = torch.load(model_save_dir+'epoch_3_score_0.88577.pth', map_location='cpu')
efnet.load_state_dict(load2['state_dict'])
_ = efnet.eval()

In [ ]:
#function which drops lateral image records and irrelevant columns, replaces -1 with 0, corrects Path column

def dropper(df):
    d = df.copy()
    index = d[d["Frontal/Lateral"] == "Lateral"].index
    d.drop(index=index, axis=0, inplace=True)
    d = d.drop(columns=['Sex','Age','Frontal/Lateral','AP/PA'])
    #d = d.replace(-1.0,0)
    d.Path = d.Path.str.replace('CheXpert-','chexpert/')
    d = d.reset_index(drop=True)
    return d

In [ ]:
val = pd.read_csv(valid_csv_path, sep=',').fillna(0)

val_df = dropper(val)

In [ ]:
val_df.head()

,Path,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,chexpert/v1.0/valid/patient64541/study1/view1_...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chexpert/v1.0/valid/patient64542/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,chexpert/v1.0/valid/patient64543/study1/view1_...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chexpert/v1.0/valid/patient64544/study1/view1_...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,chexpert/v1.0/valid/patient64545/study1/view1_...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((456, 456)),
    transforms.ToTensor(),
])

In [ ]:
# dataset and dataloader
#path = all_data['Path'], labels = all_data[all_data.columns[1:15]]

class XRayDataset(Dataset):
    
    def __init__(self, path:pd.Series, labels:pd.DataFrame, transform=None):
        self.path = path.reset_index(drop=True)
        self.labels = labels.reset_index(drop=True)
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.path[index].replace('CheXpert-', 'chexpert/')
        dir_path = '/home/tu-serbin/data'
        image = Image.open(os.path.join(dir_path, img_path)).convert('RGB')
        label = self.labels.loc[index].to_numpy()
        
        if self.transform:
            image = self.transform(image)
            
        return image, label
            
    def __len__(self):
        return self.path.shape[0]
        
        
        

In [ ]:
# dataset and dataloader


class Datasets(Dataset):
    
    def __init__(self, state, transform=None):
        self.path = state.iloc[1:,0].values
        labels = state.iloc[1:, 1:].values
        for row in range(labels.shape[0]):
            for column in range(labels.shape[1]):
                if labels[row][column] != 0.0 and labels[row][column] != 1.0:
                    labels[row][column] = round(np.random.uniform(0.55, 0.85), 2)
        self.labels = labels
        
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.path[index]
        img_path = img_path.replace('CheXpert-', 'chexpert/')
        dir_path = '/home/tu-serbin/data'
        image = Image.open(os.path.join(dir_path, img_path)).convert('RGB')
        label = self.labels[index]
        
        if self.transform:
            image = self.transform(image)
            
        return image, torch.FloatTensor(label)
            
    def __len__(self):
        return self.path.shape[0]

In [ ]:
train_data = Datasets(train_csv, train_transform)
val_data = Datasets(valid_csv, test_transform)

# put it into dataloader
batch_size = 6
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

In [ ]:
class Efficientnet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.net = EfficientNet.from_pretrained('efficientnet-b5')
        self.net._fc = nn.Sequential(nn.Linear(2048, 14), nn.Sigmoid())
        
    def forward(self,x):
        
        return self.net(x)

In [ ]:

model = Efficientnet().cuda()


#choose Loss, Optimizer
loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999))
learning_rate_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5, last_epoch=-1)

Loaded pretrained weights for efficientnet-b5


In [ ]:
labels_name = train_csv.columns[1:].values
target = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
target_index = [list(labels_name).index(item) for item in target]
target_index

[8, 2, 6, 5, 10]

In [ ]:
old = os.path.join(model_path, 'epoch_3_score_0.88577.pth' )
model.load_state_dict(torch.load(old)['state_dict'])
optimizer.load_state_dict(torch.load(old)['optimizer'])

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.001
    lr: 3.125e-05
    weight_decay: 0
)

In [ ]:
target_scores = []
target_score_averages = []
new_scores = []

# use tensorboard to record
writer = SummaryWriter()

epoch_start = 4
num_epoch = 40
best_score = 0.0
for epoch in range(epoch_start, num_epoch):
    
    epoch_star = time.time()
    train_loss = 0.0
    val_loss = 0.0
    val_pred = torch.FloatTensor().cuda()     # Tensor stores prediction values
    val_label = torch.FloatTensor().cuda()       # Tensor stores true values
    
    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        train_prd = model(data[0].cuda())
        batch_loss = loss(train_prd, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        # tensorboard for batch loss
        train_loss += batch_loss.item()
        index_train = len(train_loader) * epoch + i
        writer.add_scalar('batch_loss', batch_loss.item() , index_train)
    
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(val_loader):
            prd = model(data[0].cuda())
            batch_loss = loss(prd, data[1].cuda())

            val_loss += batch_loss.item()
            val_pred = torch.cat((val_pred, prd), 0)
            val_label = torch.cat((val_label, data[1].cuda()), 0)
      
    learning_rate_scheduler.step()
    
    #tensorboard for score and train loss
    target_score = [roc_auc_score(val_label.cpu().numpy()[:, i], val_pred.detach().cpu().numpy()[:, i],average='macro', multi_class='ovo') for i in target_index]
    target_scores.append(target_score)
    target_score_average = roc_auc_score(val_label.cpu().numpy()[:, target_index], val_pred.detach().cpu().numpy()[:, target_index],average='macro', multi_class='ovo')
    target_score_averages.append(target_score_average)
#     new_score = roc_auc_score(val_label.cpu().numpy(), val_pred.detach().cpu().numpy(),average='macro', multi_class='ovo')
#     new_scores.append(new_score)
    
    train_loss = train_loss/len(train_loader)

    best_score = target_score_average.round(5)
    torch.save({'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict()}, os.path.join(model_path, f'epoch_{epoch}_score_{best_score}.pth'))
        
    writer.add_scalars('train/val for epochs', {'train_loss':np.array(train_loss), \
                                               'val_target':np.array(target_score_average),\
                                               'val_loss': np.array(val_loss)}, epoch)


writer.close()

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.0005
    weight_decay: 0
)

In [ ]:
target_scores
[[0.8665608465608465,
  0.8032805429864254,
  0.9040310650887574,
  0.917340521114106,
  0.9275775547445255],
 [0.8707936507936508,
  0.8100678733031674,
  0.904215976331361,
  0.9101527403414195,
  0.9262089416058393],
 [0.8704761904761904,
  0.8046380090497739,
  0.9060650887573964,
  0.9122491764001197,
  0.9255246350364963],
 [0.8707936507936508,
  0.8061085972850679,
  0.9051405325443787,
  0.9126984126984126,
  0.9257527372262773]]

[[0.8665608465608465,
  0.8032805429864254,
  0.9040310650887574,
  0.917340521114106,
  0.9275775547445255],
 [0.8707936507936508,
  0.8100678733031674,
  0.904215976331361,
  0.9101527403414195,
  0.9262089416058393],
 [0.8704761904761904,
  0.8046380090497739,
  0.9060650887573964,
  0.9122491764001197,
  0.9255246350364963],
 [0.8707936507936508,
  0.8061085972850679,
  0.9051405325443787,
  0.9126984126984126,
  0.9257527372262773]]

In [ ]:
target_score_averages
[0.8837581060989322,
 0.8842878364750876,
 0.8837906199439954,
 0.8840987861095574]

[0.8837581060989322,
 0.8842878364750876,
 0.8837906199439954,
 0.8840987861095574]

In [ ]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.001
    lr: 1.953125e-06
    weight_decay: 0
)